## Court Caseload Extract

In [2]:
!pip install pydbtools
!pip install numpy==1.24.3 --user --force-reinstall
!pip install "pybind11>=2.12" -force-reinstall

  Using cached pydbtools-5.6.4-py3-none-any.whl (12 kB)
  Using cached sql_metadata-2.15.0-py3-none-any.whl (22 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl (44 kB)
  Using cached arrow_pd_parser-2.2.0-py3-none-any.whl (26 kB)
  Using cached awswrangler-3.11.0-py3-none-any.whl (379 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Using cached pyarrow-19.0.1-cp39-cp39-manylinux_2_28_x86_64.whl (42.1 MB)
  Using cached mojap_metadata-1.15.3-py3-none-any.whl (52 kB)
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Using cached pyarrow-18.1.0-cp39-cp39-manylinux_2_28_x86_64.whl (40.1 MB)
  Using cached typing_extensions-4.13.0-py3-none-any.whl (45 kB)
  Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Using cached dataengineeringutils3-1.4.3-py3-none-any.whl (12 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1

In [3]:
import pydbtools as pydb
import pandas as pd

## Quarterly Figures

In [4]:
# Query to get the dataset from athena
q_query = "Select * from crown_court_rpt_dev_dbt.receipts_disposals_open_quarterly"
dfq = pydb.read_sql_query(q_query)

#Get the first two rows of the dataset to explore
dfq.head(2)

,year,quarter,receipts_all,disposals_all,open_cases_all,receipts_trial,disposals_trial,open_cases_trial,receipts_sentence,disposals_sentence,open_cases_sentence,receipts_appeal,disposals_appeal,open_cases_appeal,receipts_unknown,disposals_unknown,open_cases_unknown
0,2016,January to March,28680,32100,48213,18677,21962,40582,7486,7507,4848,2517,2631,2783,0,0,0
1,2016,April to June,27763,31040,44815,17601,20701,37396,7595,7656,4757,2567,2683,2662,0,0,0


## Rename Columns To Conform with Output specification 

Get Column Names from dbt config

In [5]:
!pip install pyyaml
import yaml

In [6]:
#Import DBT config file

file_path = r'crown_court_rpt__receipts_disposals_open_quarterly.yml'
with open(file_path, 'r') as file:
    data = yaml.safe_load(file)
print(data)

{'version': 2, 'models': [{'name': 'crown_court_rpt__receipts_disposals_open_quarterly', 'description': 'This model represents the base component of the reporting layer, utilizing data from the OneCrown dimensional model to create a quarterly summary of the caseload report', 'columns': [{'name': 'year', 'description': 'Year'}, {'name': 'quarter', 'description': 'Quarter'}, {'name': 'receipts_all', 'description': 'All cases:receipts'}, {'name': 'disposals_all', 'description': 'All cases:disposals'}, {'name': 'open_cases_all', 'description': 'All cases:open'}, {'name': 'receipts_trial', 'description': 'All trials:receipts'}, {'name': 'disposals_trial', 'description': 'All trials:disposals'}, {'name': 'open_cases_trial', 'description': 'All trials:open'}, {'name': 'receipts_sentence', 'description': 'Committed for sentence:receipts'}, {'name': 'disposals_sentence', 'description': 'Committed for sentence:disposals'}, {'name': 'open_cases_sentence', 'description': 'Committed for sentence:op

In [7]:
#Get Output Column Names from .yml description tags

updated_column_name = [] 
for model in data['models']:
    if 'description' in model:
        updated_column_name.append(model['description'])
    if 'columns' in model:
        for column in model['columns']:
            if 'description' in column:
                updated_column_name.append(column['description'])
del updated_column_name[:1]
print(updated_column_name) 

['Year', 'Quarter', 'All cases:receipts', 'All cases:disposals', 'All cases:open', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Committed for sentence:receipts', 'Committed for sentence:disposals', 'Committed for sentence:open', 'Appeals:receipts', 'Appeals:disposals', 'Appeals:open', 'Unknown:receipts', 'Unknown:disposals', 'Unknown:open']


In [8]:
#Counts to check column (items) in list

count = len(updated_column_name)
print(count)

17


In [9]:
#Map Dataframe columns to updated columns from list

#dfq = dfq.drop(columns='id')
dfq.columns = updated_column_name
column_list = dfq.columns.to_list()
print(column_list)

['Year', 'Quarter', 'All cases:receipts', 'All cases:disposals', 'All cases:open', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Committed for sentence:receipts', 'Committed for sentence:disposals', 'Committed for sentence:open', 'Appeals:receipts', 'Appeals:disposals', 'Appeals:open', 'Unknown:receipts', 'Unknown:disposals', 'Unknown:open']


# Build GPTables for Quarterly figures

In [10]:
!pip install gptables

import gptables as gpt

  Using cached gptables-1.2.0-py3-none-any.whl (72 kB)
  Using cached XlsxWriter-3.2.2-py3-none-any.whl (165 kB)


In [11]:
# Define Metadata for the quarterly tables 

metadata = {
    "title": "Table C1: Receipts, disposals and open criminal cases in the Crown Court in England and Wales, annually 2016 - 2023, quarterly Q1 2016 - Q3 2024 [note 13][note 14][note 15][note 16][note 118] (“One Crown”)",
    "source": "Source: XHIBIT system and Common Platform, HMCTS",
}

#Create a GPTable object

q_table = gpt.GPTable(
    table=dfq,
    index_columns={1:0},
    title=metadata["title"],
    table_name="Crown_Court_Cases",
   ## subtitle=metadata["subtitle"],
    source=metadata["source"],
)

q_data = q_table.table

#Convert GPTable to Dataframe 
dfql = pd.DataFrame(q_data)



# Yearly Figures

In [12]:
# Query to get the dataset from athena

y_query = "Select * from crown_court_rpt_dev_dbt.receipts_disposals_open_annual"
dfy = pydb.read_sql_query(y_query)

#Get the first two rows of the dataset to explore

dfy.head(2)

,year,receipts_all,disposals_all,open_cases_all,receipts_trial,disposals_trial,open_cases_trial,receipts_sentence,disposals_sentence,open_cases_sentence,receipts_appeal,disposals_appeal,open_cases_appeal,receipts_unknown,disposals_unknown,open_cases_unknown
0,2016,112005,120143,43062,71433,79710,35329,30108,29805,5012,10464,10628,2721,0,0,0
1,2017,109353,113029,38922,68439,72018,31454,31053,30828,5082,9861,10183,2386,0,0,0


# Rename To Conform with Output specification

In [13]:
#Import DBT config file

file_path_y = r'crown_court_rpt__receipts_disposals_open_annual.yml'
with open(file_path_y, 'r') as file:
    data_y = yaml.safe_load(file)
print(data_y)

{'version': 2, 'models': [{'name': 'crown_court_rpt__receipts_disposals_open_annual', 'description': 'This model represents the base component of the reporting layer, utilizing data from the OneCrown dimensional model to create a annual summary of the caseload report', 'columns': [{'name': 'year', 'description': 'Year'}, {'name': 'receipts_all', 'description': 'All cases:receipts'}, {'name': 'disposals_all', 'description': 'All cases:disposals'}, {'name': 'open_cases_all', 'description': 'All cases:open'}, {'name': 'receipts_trial', 'description': 'All trials:receipts'}, {'name': 'disposals_trial', 'description': 'All trials:disposals'}, {'name': 'open_cases_trial', 'description': 'All trials:open'}, {'name': 'receipts_sentence', 'description': 'Committed for sentence:receipts'}, {'name': 'disposals_sentence', 'description': 'Committed for sentence:disposals'}, {'name': 'open_cases_sentence', 'description': 'Committed for sentence:open'}, {'name': 'receipt_appeals', 'description': 'App

In [14]:
#Get Output Column Names from .yml description tags

updated_column_name_y = [] 
for model in data_y['models']:
    if 'description' in model:
        updated_column_name_y.append(model['description'])
    if 'columns' in model:
        for column in model['columns']:
            if 'description' in column:
                updated_column_name_y.append(column['description'])
del updated_column_name_y[:1]
print(updated_column_name_y) 

['Year', 'All cases:receipts', 'All cases:disposals', 'All cases:open', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Committed for sentence:receipts', 'Committed for sentence:disposals', 'Committed for sentence:open', 'Appeals:receipts', 'Appeals:disposals', 'Appeals:open', 'Unknown:receipts', 'Unknown:disposals', 'Unknown:opens']


In [15]:
#Counts to check column (items) in list

count = len(updated_column_name_y)
print(count)

16


In [16]:
#Map Dataframe columns to updated columns from list

#dfy = dfy.drop(columns='id')
dfy.columns = updated_column_name_y
column_list_y = dfy.columns.to_list()
print(column_list_y)

['Year', 'All cases:receipts', 'All cases:disposals', 'All cases:open', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Committed for sentence:receipts', 'Committed for sentence:disposals', 'Committed for sentence:open', 'Appeals:receipts', 'Appeals:disposals', 'Appeals:open', 'Unknown:receipts', 'Unknown:disposals', 'Unknown:opens']


# Build GPtable for Yearly Figures

In [17]:
# Define Metadata for the quarterly tables 

metadata = {
    "title": "Table C1: Receipts, disposals and open criminal cases in the Crown Court in England and Wales, annually 2016 - 2023, quarterly Q1 2016 - Q3 2024 [note 13][note 14][note 15][note 16][note 118] (“One Crown”)",
    "subtitle1": "This worksheet contains one table.",
    "subtitle2": "This table contains notes, which can be found in the Notes worksheet.",
    "source": "Source: XHIBIT system and Common Platform, HMCTS",
}

#Create a GPTable object

y_table = gpt.GPTable(
    table=dfy,
    index_columns={1:0},
    title=metadata["title"],
    table_name="Crown_Court_Cases",
   ## subtitle=metadata["subtitle"],
    source=metadata["source"],
)
y_data = y_table.table

#Convert GPTable to Dataframe
dfyl = pd.DataFrame(y_data)

## Trial Quarterly Figures

In [75]:
# Query to get the dataset from athena
tq_query = "Select * from crown_court_rpt_dev_dbt.trial_receipts_disposals_open_quarterly order by year"
dfqt = pydb.read_sql_query(tq_query)

#Get the first two rows of the dataset to explore
dfqt.head(2)

,year,quarter,receipts_trial_all,disposals_trial_all,open_cases_trial_all,receipts_trial_tew,disposals_trial_tew,open_cases_trial_tew,receipts_trial_io,disposals_trial_io,open_cases_trial_io,receipts_trial_unknown,disposals_trial_unknown,open_cases_trial_unknown
0,2016,January to March,18677,21962,40582,12359,15063,25268,6318,6899,15314,0,0,0
1,2016,April to June,17601,20701,37396,11321,13685,22880,6280,7016,14516,0,0,0


In [76]:
#Import DBT config file

file_path_tq = r'crown_court_rpt__trial_receipts_disposals_open_quarterly.yml'
with open(file_path_tq, 'r') as file:
    data_t = yaml.safe_load(file)
print(data_t)

{'version': 2, 'models': [{'name': 'crown_court_rpt__trial_receipts_disposals_open_quarterly', 'description': 'This model represents the base component of the reporting layer, utilizing data from the OneCrown dimensional model to create a quarterly summary of the caseload report', 'columns': [{'name': 'year', 'description': 'Year'}, {'name': 'quarter', 'description': 'Quarter'}, {'name': 'receipts_trial_all', 'description': 'All trials:receipts'}, {'name': 'disposals_trial_all', 'description': 'All trials:disposals'}, {'name': 'open_cases_trial_all', 'description': 'All trials:open'}, {'name': 'receipts_trial_tew', 'description': 'Triable-either-way trials:receipts'}, {'name': 'disposals_trial_tew', 'description': 'Triable-either-way trials:disposals'}, {'name': 'open_cases_trial_tew', 'description': 'Triable-either-way trials:open'}, {'name': 'receipts_trial_io', 'description': 'Indictable only trials:receipts'}, {'name': 'disposals_trial_io', 'description': 'Indictable only trials:di

In [77]:
#Get Output Column Names from .yml description tags

updated_column_name_tq = [] 
for model in data_t['models']:
    if 'description' in model:
        updated_column_name_tq.append(model['description'])
    if 'columns' in model:
        for column in model['columns']:
            if 'description' in column:
                updated_column_name_tq.append(column['description'])
del updated_column_name_tq[:1]
print(updated_column_name_tq) 

['Year', 'Quarter', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Triable-either-way trials:receipts', 'Triable-either-way trials:disposals', 'Triable-either-way trials:open', 'Indictable only trials:receipts', 'Indictable only trials:disposals', 'Indictable only trials:open', 'Trial Unknown:receipts', 'Trial Unknown:disposals', 'Trial Unknown:Open Cases']


In [78]:
#Counts to check column (items) in list

count = len(updated_column_name_tq)
print(count)

14


In [79]:
#Map Dataframe columns to updated columns from list

#dfq = dfq.drop(columns='id')
dfqt.columns = updated_column_name_tq
column_list_tq = dfqt.columns.to_list()
print(column_list_tq)

['Year', 'Quarter', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Triable-either-way trials:receipts', 'Triable-either-way trials:disposals', 'Triable-either-way trials:open', 'Indictable only trials:receipts', 'Indictable only trials:disposals', 'Indictable only trials:open', 'Trial Unknown:receipts', 'Trial Unknown:disposals', 'Trial Unknown:Open Cases']


In [83]:
# Define Metadata for the quarterly tables 

metadata = {
    "title": "Table C1: Receipts, disposals and open criminal cases in the Crown Court in England and Wales, annually 2016 - 2023, quarterly Q1 2016 - Q3 2024 [note 13][note 14][note 15][note 16][note 118] (“One Crown”)",
    "subtitle1": "This worksheet contains one table.",
    "subtitle2": "This table contains notes, which can be found in the Notes worksheet.",
    "source": "Source: XHIBIT system and Common Platform, HMCTS",
}

#Create a GPTable object

tq_table = gpt.GPTable(
    table=dfqt,
    index_columns={1:0},
    title=metadata["title"],
    table_name="Crown_Court_Cases",
   ## subtitle=metadata["subtitle"],
    source=metadata["source"],
)

tq_data = tq_table.table

#Convert GPTable to Dataframe 
dfqlt = pd.DataFrame(tq_data)


## Trial Yearly Figures

In [51]:
# Query to get the dataset from athena
ty_query = "Select * from crown_court_rpt_dev_dbt.trial_receipts_disposals_open_annual"
dfyt = pydb.read_sql_query(ty_query)

#Get the first two rows of the dataset to explore
dfyt.head(2)

,year,receipts_trial_all,disposals_trial_all,open_cases_trial_all,receipts_trial_tew,disposals_trial_tew,open_cases_trial_tew,receipts_trial_io,disposals_trial_io,open_cases_trial_io,receipts_trial_unknown,disposals_trial_unknown,open_cases_trial_unknown
0,2016,71433,79710,35329,46515,52944,21414,24918,26766,13915,0,0,0
1,2017,68439,72018,31454,43759,46543,18450,24680,25475,13004,0,0,0


In [52]:
#Import DBT config file

file_path_ta = r'crown_court_rpt__trial_receipts_disposals_open_annual.yml'
with open(file_path_ta, 'r') as file:
    data_ta = yaml.safe_load(file)
print(data_ta)

{'version': 2, 'models': [{'name': 'crown_court_rpt__trial_receipts_disposals_open_annual', 'description': 'This model represents the base component of the reporting layer, utilizing data from the OneCrown dimensional model to create a annual summary of the caseload report', 'columns': [{'name': 'year', 'description': 'Year'}, {'name': 'receipts_trial_all', 'description': 'All trials:receipts'}, {'name': 'disposals_trial_all', 'description': 'All trials:disposals'}, {'name': 'open_cases_trial_all', 'description': 'All trials:open'}, {'name': 'receipts_trial_tew', 'description': 'Triable-either-way trials:receipts'}, {'name': 'disposals_trial_tew', 'description': 'Triable-either-way trials:disposals'}, {'name': 'open_cases_trial_tew', 'description': 'Triable-either-way trials:open'}, {'name': 'receipts_trial_io', 'description': 'Indictable only trials:receipts'}, {'name': 'disposals_trial_io', 'description': 'Indictable only trials:disposals'}, {'name': 'open_cases_trial_io', 'descripti

In [53]:
#Get Output Column Names from .yml description tags

updated_column_name_ta = [] 
for model in data_ta['models']:
    if 'description' in model:
        updated_column_name_ta.append(model['description'])
    if 'columns' in model:
        for column in model['columns']:
            if 'description' in column:
                updated_column_name_ta.append(column['description'])
del updated_column_name_ta[:1]
print(updated_column_name_ta) 

['Year', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Triable-either-way trials:receipts', 'Triable-either-way trials:disposals', 'Triable-either-way trials:open', 'Indictable only trials:receipts', 'Indictable only trials:disposals', 'Indictable only trials:open', 'Trial Unknown:receipts', 'Trial Unknown:disposals', 'Trial Unknown:Open Cases']


In [54]:
#Counts to check column (items) in list

ta_count = len(updated_column_name_ta)
print(ta_count)

13


In [55]:
#Map Dataframe columns to updated columns from list

#dfq = dfq.drop(columns='id')
dfyt.columns = updated_column_name_ta
column_list_ta = dfyt.columns.to_list()
print(column_list_ta)

['Year', 'All trials:receipts', 'All trials:disposals', 'All trials:open', 'Triable-either-way trials:receipts', 'Triable-either-way trials:disposals', 'Triable-either-way trials:open', 'Indictable only trials:receipts', 'Indictable only trials:disposals', 'Indictable only trials:open', 'Trial Unknown:receipts', 'Trial Unknown:disposals', 'Trial Unknown:Open Cases']


In [62]:
# Define Metadata for the quarterly tables 

metadata = {
    "title": "Table C1: Receipts, disposals and open criminal cases in the Crown Court in England and Wales, annually 2016 - 2023, quarterly Q1 2016 - Q3 2024 [note 13][note 14][note 15][note 16][note 118] (“One Crown”)",
    "subtitle1": "This worksheet contains one table.",
    "subtitle2": "This table contains notes, which can be found in the Notes worksheet.",
    "source": "Source: XHIBIT system and Common Platform, HMCTS",
}

#Create a GPTable object

ta_table = gpt.GPTable(
    table=dfyt,
    index_columns={1:0},
    title=metadata["title"],
    table_name="Crown_Court_Cases",
   ## subtitle=metadata["subtitle"],
    source=metadata["source"],
)

ta_data = ta_table.table

#Convert GPTable to Dataframe 
dfylt = pd.DataFrame(ta_data)


# Export Quarterly and Yearly Figures to Excel Document

In [59]:
!pip install openpyxl
!pip install XlsxWriter

import openpyxl
import xlsxwriter
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment

In [89]:
# Save the table as an Excel File 

output_filename = "crown_court_extract.xlsx"
with pd.ExcelWriter(output_filename, engine="openpyxl") as writer:
    dfyl.to_excel(writer, sheet_name="Yearly_Crown_Court_Cases", startrow=5, index=False)
    dfql.to_excel(writer, sheet_name="Quarterly_Crown_Court_Cases", startrow=5, index=False)
    dfylt.to_excel(writer, sheet_name="Trial_Yearly_Crown_Court_Cases", startrow=5, index=False)
    dfqlt.to_excel(writer, sheet_name="Trial_Quarterly_Crown_Court_Cases", startrow=5, index=False)

# Function To Adjust Column width
def auto_adjust_column_width(ws, column_letter):
    max_length = 0
    col_index = ws[column_letter]  # Get the entire column

    for cell in col_index:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))

    adjusted_width = max_length + 2  # Add some padding
    ws.column_dimensions[column_letter].width = adjusted_width
    
    
#Edit WorkBook

#Load Workbook
wb = load_workbook(output_filename)


def insert_metadata(sheet_name):
    ws = wb[sheet_name]
    ws["A1"] = metadata["title"]  # Title in first row
    ws["A2"] = metadata["subtitle1"]  # subtitle1 in second row
    ws["A3"] = metadata["subtitle2"]  # subtitle2 in third row
    ws["A4"] = metadata["source"]  # Source in fourth row
    
    ws["A1"].font = Font(bold=True)  # Make title bold
    
    # Apply wrap text to row 4 (column headers)
    for col in range(1, ws.max_column + 1):  # Loop through all columns
        ws.cell(row=6, column=col).alignment = Alignment(wrap_text=True)

# Apply metadata to both sheets
insert_metadata("Yearly_Crown_Court_Cases")
insert_metadata("Quarterly_Crown_Court_Cases")
insert_metadata("Trial_Yearly_Crown_Court_Cases")
insert_metadata("Trial_Quarterly_Crown_Court_Cases")

## Auto Adjust Quarter column
wsq = wb["Quarterly_Crown_Court_Cases"]  # Get the worksheet
auto_adjust_column_width(wsq, "B")

wsqt = wb["Trial_Quarterly_Crown_Court_Cases"]  # Get the worksheet
auto_adjust_column_width(wsqt, "B")

# Save the updated file
wb.save(output_filename)

## Download Link 

In [90]:
from IPython.display import HTML

def create_download_link(filename):
    return HTML(f'<a href="{filename}" download>Click here to download {filename}</a>')

create_download_link(output_filename)

Copy Workbook To S3 Location

In [53]:
!pip install boto3

In [56]:
import boto3

In [58]:
# Set up AWS session with credentials
#session = boto3.Session(
#    aws_access_key_id="",
#    aws_secret_access_key="",
#    region_name=""
#)

# Create an S3 client using the session
s3_client = boto3.client("s3")

# Prepare the file and the bucket name
bucket_name = "alpha-fotest"  # Enclosed in quotes
output_filename = "crown_court_extract.xlsx"  # The file you're uploading
data = open(output_filename, "rb")  # Open the file in binary mode

response = s3_client.put_object(Bucket=bucket_name , Body=data, Key=output_filename)
print(f"AWS response code for uploading file is {(response['ResponseMetadata']['HTTPStatusCode'])}")

AWS response code for uploading file is 200
